In [1]:
import os
import json
from openai import OpenAI
client = OpenAI()

client.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
def get_current_weather(location, unit="섭씨"):
    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info, ensure_ascii=False)


In [ ]:
assistant = client.beta.assistants.create(
    name="날씨봇",
    instructions="당신은 날씨 챗봇입니다. 사용자의 질문에 대답하기 위해 주어진 function들을 사용하세요.",
    tools = [
        {
            "type": "function",
            "function":
            {
                "name": "get_current_weather",
                "description": "특정 지역의 날씨를 알려줍니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "지역이름. 예) 서울, 부산, 제주도",
                        },
                        "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
                    },
                    "required": ["location"],
                },
            }
        }
    ],
    model="gpt-4o-mini"
)
assistant

Assistant(id='asst_MsXyUbEdrb88NwEJv19EpX8U', created_at=1710317535, description=None, file_ids=[], instructions='당신은 날씨 챗봇입니다. 사용자의 질문에 대답하기 위해 주어진 function들을 사용하세요.', metadata={}, model='gpt-3.5-turbo', name='날씨봇', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='get_current_weather', description='특정 지역의 날씨를 알려줍니다.', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': '지역이름. 예) 서울, 부산, 제주도'}, 'unit': {'type': 'string', 'enum': ['섭씨', '화씨']}}, 'required': ['location']}), type='function')])

In [4]:
thread = client.beta.threads.create()
thread

Thread(id='thread_IFBS5vVnbulLg9P1erjjk2bh', created_at=1710317562, metadata={}, object='thread')

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="서울의 날씨는 어때?"
)

In [6]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

[ThreadMessage(id='msg_f3gYb0kx0Dhax2gEVbDivr1t', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='서울의 날씨는 어때?'), type='text')], created_at=1710317573, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_IFBS5vVnbulLg9P1erjjk2bh')]


In [7]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [8]:
run_id = run.id

In [9]:
import time

while True:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run_id
    )
    if run.required_action is not None:
        break

In [10]:
print(run.required_action.submit_tool_outputs.tool_calls[0])

RequiredActionFunctionToolCall(id='call_kz6dTdxLM4p7WqnB1jBcDwob', function=Function(arguments='{"location":"서울","unit":"섭씨"}', name='get_current_weather'), type='function')


In [11]:
import json

available_functions = {
    "get_current_weather": get_current_weather,
}

for tool_call in run.required_action.submit_tool_outputs.tool_calls:
    tool_call_id = tool_call.id
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args['location']
    )

print(function_response)

{"location": "서울", "temperature": "24", "unit": "섭씨", "forecast": ["sunny", "windy"]}


In [12]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
      {
        "tool_call_id": tool_call_id,
        "output": json.dumps(function_response, ensure_ascii=False)
      }
    ]
)
run_id = run.id

In [13]:
import time

while run.status != "completed"  :
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    time.sleep(0.1)

In [14]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [15]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data[0].content[0].text.value)

서울의 현재 날씨는 24°C이며 맑고 바람이 많이 불고 있습니다.
